In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
in_path  = '/content/drive/MyDrive/MRP/initial_merged_prices.csv'
out_path = '/content/drive/MyDrive/MRP/price_preprocessed.csv'

In [3]:
# Load the merged price data
df = pd.read_csv(in_path, parse_dates=['date'])
print("Original shape:", df.shape)
df.head()

Original shape: (29677722, 8)


,date,volume,open,high,low,close,adj close,symbol
0,1999-11-18,62546300.0,32.546494,35.765381,28.612303,31.473534,27.068665,A
1,1999-11-19,15234100.0,30.713520,30.758226,28.478184,28.880543,24.838577,A
2,1999-11-22,6577800.0,29.551144,31.473534,28.657009,31.473534,27.068665,A
3,1999-11-23,5975600.0,30.400572,31.205294,28.612303,28.612303,24.607880,A
4,1999-11-24,4843200.0,28.701717,29.998211,28.612303,29.372318,25.261524,A


In [4]:
# Keep only prices from Jan 1, 2016 through Dec 31, 2023
start, end = '2016-01-01', '2023-12-31'
mask = df['date'].between(start, end)
df = df[mask].reset_index(drop=True)

print("Filtered to 2016–2023 -> new shape:", df.shape)

Filtered to 2016–2023 -> new shape: (11097773, 8)


In [5]:
# Drop the rows with any nulls
df.dropna(inplace=True)
print("After dropna:", df.shape)

After dropna: (11097716, 8)


In [6]:
# Remove zeros and extreme outlier prices
mask = (
    (df[['open','high','low','close','adj close']] > 0).all(axis=1) &
    (df['open'] < 1e4)  # cap based on domain knowledge
)
df = df[mask]
print("After outlier filtering:", df.shape)

After outlier filtering: (11090187, 8)


In [7]:
# Drop exact duplicates on (symbol, date)
df.drop_duplicates(subset=['symbol','date'], inplace=True)
print("After dropping duplicates:", df.shape)

After dropping duplicates: (11090187, 8)


In [8]:
# Keep only symbols with >= 100 trading days
counts = df['symbol'].value_counts()
valid = counts[counts >= 100].index
df = df[df['symbol'].isin(valid)]
print("After filtering short histories:", df.shape)

After filtering short histories: (11083232, 8)


In [9]:
# Sort by symbol & date, then reset index
df.sort_values(['symbol','date'], inplace=True)
df.reset_index(drop=True, inplace=True)
print("After sort & reset_index:", df.shape)

After sort & reset_index: (11083232, 8)


In [10]:
df.head()

,date,volume,open,high,low,close,adj close,symbol
0,2016-01-04,3287300.0,41.060001,41.189999,40.340000,40.689999,39.089256,A
1,2016-01-05,2587200.0,40.730000,40.950001,40.340000,40.549999,38.954754,A
2,2016-01-06,2103600.0,40.240002,40.990002,40.049999,40.730000,39.127674,A
3,2016-01-07,3504300.0,40.139999,40.150002,38.810001,39.000000,37.465733,A
4,2016-01-08,3736700.0,39.220001,39.709999,38.470001,38.590000,37.071869,A


In [12]:
import numpy as np

# Cap or log-transform extreme volume
df['volume_capped'] = df['volume'].clip(upper=df['volume'].quantile(0.99))
df['log_volume'] = np.log1p(df['volume_capped'])  # smoothed log transformation

In [13]:
# Save the preprocessed file
df.to_csv(out_path, index=False)
print("Saved cleaned dataset to:", out_path)

Saved cleaned dataset to: /content/drive/MyDrive/MRP/price_preprocessed.csv
